In [1]:
import cobra.test
import os
from os.path import join
from cobra.io import read_sbml_model
model = cobra.io.load_json_model(join("RehMBEL1391_sbml_L3V1.json"))
from cobra import Model, Reaction, Metabolite

# Adding glycerol to the medium

**Current carbon source: Fructose**

**Model information:**

In [2]:
model

Name,RehMBEL1391
Memory address,0x07f877e096c70
Number of metabolites,1348
Number of reactions,1538
Number of groups,0
Objective expression,1.0*Biomass - 1.0*Biomass_reverse_57a34
Compartments,"c, e"


**Uptake and secrection of raw model**

In [3]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
fe2_e,EX_fe2_e,6.333E-05,0,0.00%
fru_e,EX_fru_e,3,6,100.00%
h_e,EX_h_e,10.72,0,0.00%
nh4_e,EX_nh4_e,3.038,0,0.00%
o2_e,EX_o2_e,5.734,0,0.00%
pi_e,EX_pi_e,7.987,0,0.00%
so4_e,EX_so4_e,0.05797,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
BIOMASS_c,EX_BIOMASS_c,-0.2853,0,0.00%
co2_e,EX_co2_e,-6.151,1,100.00%


## Checking if the model can currently uptake glycerol 

In [4]:
for metabolite in model.metabolites.query('glycerin', 'name'):
    print(metabolite.name)

In [5]:
for metabolite in model.metabolites.query('glycerol', 'name'):
    print(metabolite.name)

CDP-diacylglycerol
D-erythro-1-(Imidazol-4-yl)glycerol 3-phosphate
Triacylglycerol
1,2-Diacyl-sn-glycerol
Phosphatidylglycerol
Glycerophosphoglycerol
1-Acyl-sn-glycerol 3-phosphate
1-Acylglycerol
C1-(3-Indolyl)-glycerol 3-phosphate


-> No glycerol uptake pathway in the current model

## Adding a new glycerolipid pathway

### First reaction: Glycerol dehydrogenase 

Check if the metabolites of this reaction already exist in the model. If not adding missing metabolites.

In [6]:
print("Check glycerol")
for metabolite in model.metabolites.query('glycerol', 'name'):
    print(metabolite.name)
model.add_metabolites([Metabolite("glycerol_c", formula='C3H8O3', name="glycerol_c", compartment = "c")])
print("____________________________")

print("Check NADP+")
for metabolite in model.metabolites.query('nadp', 'name'):
    print(metabolite.name)
print(model.metabolites.nadp_c)
print("____________________________")

### Check for D-glyceraldehyde
print("Check D-glyceraldehyde")
for metabolite in model.metabolites.query('d-glyceraldehyde', 'name'):
    print(metabolite.name)
model.add_metabolites([Metabolite("glyceraldehyde_c", formula='C3H6O3', name="glyceraldehyde_c", compartment = "c")])
print("____________________________")

### Check for NADPH
print("Check NADPH")
for metabolite in model.metabolites.query('nadph', 'name'):
    print(metabolite.name)
print(model.metabolites.nadph_c)
print("____________________________")

### Check for  H+
print("Check H+")
for metabolite in model.metabolites.query('h_c', 'name'):
    print(metabolite.name)



Check glycerol
CDP-diacylglycerol
D-erythro-1-(Imidazol-4-yl)glycerol 3-phosphate
Triacylglycerol
1,2-Diacyl-sn-glycerol
Phosphatidylglycerol
Glycerophosphoglycerol
1-Acyl-sn-glycerol 3-phosphate
1-Acylglycerol
C1-(3-Indolyl)-glycerol 3-phosphate
____________________________
Check NADP+
nadp_c
____________________________
Check D-glyceraldehyde
____________________________
Check NADPH
nadph_c
____________________________
Check H+


Add reaction glycerol dehydrogenase: Glycerol + NADP+ = D-glyceraldehyde + NADPH + H+

In [7]:
Glycerol_dehydrogenase = Reaction('Gly_deh') 
Glycerol_dehydrogenase.add_metabolites({model.metabolites.glycerol_c: -1,
                            model.metabolites.nadp_c: -1,
                             model.metabolites.glyceraldehyde_c: 1,
                          model.metabolites.nadph_c: 1,
                            model.metabolites.h_c: 1, 
                              })
print(Glycerol_dehydrogenase.build_reaction_string())
model.add_reactions([Glycerol_dehydrogenase])
model.reactions.Gly_deh


glycerol_c + nadp_c --> glyceraldehyde_c + h_c + nadph_c


Reaction identifier,Gly_deh
Name,
Memory address,0x07f878038d250
Stoichiometry,glycerol_c + nadp_c --> glyceraldehyde_c + h_c + nadph_c glycerol_c + Nicotinamide adenine dinucleotide phosphate --> glyceraldehyde_c + H+ + Nicotinamide adenine dinucleotide phosphate -reduced
GPR,
Lower bound,0.0
Upper bound,1000.0


In [8]:
#model.metabolites.glycerol_c

### Add glycerol as an exchange reaction in to the model 

In [9]:
model.add_metabolites([Metabolite("glycerol_e", formula='C3H8O3', name="glycerol_e", compartment = "e")])
model.add_boundary(model.metabolites.get_by_id("glycerol_e"), type="exchange")
Glycerol_import = Reaction("glyc_imp")
Glycerol_import.add_metabolites({model.metabolites.glycerol_e: -1, model.metabolites.glycerol_c: 1})
model.add_reactions([Glycerol_import])

In [10]:
Glycerol_import.lower_bound = - 3 

In [11]:
#model.reactions.glyc_imp

In [12]:
#model.summary()
#model.reactions.EX_fru_e

### Second reaction: Aldehyde dehydrogenase

Check if the metabolites of this reaction already exist in the model. If not adding missing metabolites.

In [13]:
print("Check aldehyde")
for metabolite in model.metabolites.query('aldehyde', 'name'):
    print(metabolite.name)
model.add_metabolites([Metabolite("aldehyde_c", formula='CHOR', name="aldehyde_c", compartment = "c")])
print("____________________________")

print("Check NAD+")
for metabolite in model.metabolites.query('nad', 'name'):
    print(metabolite.name)
print(model.metabolites.nad_c)
print("____________________________")

print("Check H2O")
for metabolite in model.metabolites.query('H2O', 'name'):
    print(metabolite.name)
print("____________________________")

print("Check carboxylate")
for metabolite in model.metabolites.query('carboxylate', 'name'):
    print(metabolite.name)
print("____________________________")

### Check for NADH
print("Check NADH")
for metabolite in model.metabolites.query('nadh', 'name'):
    print(metabolite.name)
print(model.metabolites.nadh_c)
print("____________________________")

### H+ already found in glycerol dehydrogenase reaction


Check aldehyde
5-Chloro-2-hydroxymuconic semialdehyde
3,4-Dihydroxymandelaldehyde
Phenylacetaldehyde
Betaine aldehyde
L-Glutamate 1-semialdehyde
D-Glyceraldehyde 3-phosphate
Methylimidazole acetaldehyde
Imidazole-4-acetaldehyde
4-Carboxy-2-hydroxymuconate semialdehyde
Sulfoacetaldehyde
2-Amino-3-carboxymuconate semialdehyde
L-Aspartate 4-semialdehyde
Indole-3-acetaldehyde
Gentisate aldehyde
3-Chloro-2-hydroxymuconic semialdehyde
beta-Aminopropion aldehyde
Aminoacetaldehyde
Fluoroacetaldehyde
Acetaldehyde
Acetaldehyde
3-Methoxy-4-hydroxyphenylacetaldehyde
(Z)-4-Hydroxyphenylacetaldehyde-oxime
L-4-Hydroxyglutamate semialdehyde
N-Acetyl-L-glutamate 5-semialdehyde
L-Glutamate 5-semialdehyde
2-Naphthaldehyde
3-Chloroallyl aldehyde
L-Lactaldehyde
Succinate semialdehyde-thiamin diphosphate anion
Glycolaldehyde
4-Hydroxymethylsalicylaldehyde
D-Lactaldehyde
2-Hydroxy-5-methyl-cis,cis-muconic semialdehyde
2-Aminomuconate semialdehyde
Malonate semialdehyde
Succinic semialdehyde
4-Hydroxybenzaldeh

Add reaction aldehyde dehydrogenase: Aldehyde + NAD+ + H2O = carboxylate + NADH + H+

In [14]:
Oxidoreductase = Reaction("Oxi_red") 
Oxidoreductase.add_metabolites({model.metabolites.aldehyde_c: -1,
                            model.metabolites.nad_c: -1,
                            model.metabolites.h2o_c: -1,
                          model.metabolites.carboxylate_c: 1,
                            model.metabolites.nadh_c: 1, 
                              model.metabolites.h_c: 1,
                              })
print(Oxidoreductase.build_reaction_string())
model.add_reactions([Oxidoreductase])
model.reactions.Oxi_red

aldehyde_c + h2o_c + nad_c --> carboxylate_c + h_c + nadh_c


Reaction identifier,Oxi_red
Name,
Memory address,0x07f87803ad670
Stoichiometry,aldehyde_c + h2o_c + nad_c --> carboxylate_c + h_c + nadh_c aldehyde_c + H2O + Nicotinamide adenine dinucleotide --> carboxylate + H+ + Nicotinamide adenine dinucleotide -reduced
GPR,
Lower bound,0.0
Upper bound,1000.0


### Third reaction: D-glycerate 3-kinase

Check if the metabolites of this reaction already exist in the model. If not adding missing metabolites.

In [15]:
print("Check ATP")
for metabolite in model.metabolites.query('atp', 'name'):
    print(metabolite.name)
print(model.metabolites.get_by_id('atp_c'))
print("____________________________")

print("Check glycerate")
for metabolite in model.metabolites.query('glycerate', 'name'):
    print(metabolite.name)
model.add_metabolites([Metabolite("glycerate_c",formula='C3H6O4', name="glycerate_c", compartment = "c")])
print("____________________________")

print("Check ADP")
for metabolite in model.metabolites.query('ADP', 'name'):
    print(metabolite.name)
print("____________________________")

print("Check 3-phospho-D-glycerate")
for metabolite in model.metabolites.query('glycerate', 'name'):
    print(metabolite.name)
## check name in model
print(model.metabolites.get_by_id('3pg_c'))

Check ATP
atp_c
____________________________
Check glycerate
3-Phospho-D-glycerate
3-Phospho-D-glycerate
____________________________
Check ADP
ADP-D-ribose
dADP
ADP-L-glycero-D-manno-heptose
ADP-D-glycero-D-manno-heptose
ADP
____________________________
Check 3-phospho-D-glycerate
3-Phospho-D-glycerate
3-Phospho-D-glycerate
glycerate_c
3pg_c


Add reaction glycerate 3-kinase: ATP + D-glycerate = ADP + 3-phospho-D-glycerate

In [16]:
Transferases = Reaction('Tra_fer')  
Transferases.add_metabolites({model.metabolites.atp_c: -1,
                            model.metabolites.glycerate_c: -1,
                            model.metabolites.adp_c: 1,
                           model.metabolites.get_by_id('3pg_c'): 1
                              })
print(Transferases.build_reaction_string())
model.add_reactions([Transferases])

atp_c + glycerate_c --> 3pg_c + adp_c


# Comparison of biomass, PHB production rate and theoretical max. yield between fructose and glycerol in medium

Displaying current media components 

In [17]:
medium = model.medium
medium

{'EX_fe2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_pi_e': 100.0,
 'EX_cobalt2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_k_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_fru_e': 3.0,
 'EX_nh4_e': 10.0,
 'EX_na_e': 10.0,
 'EX_o2_e': 18.5,
 'EX_mobd_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 100.0,
 'EX_glycerol_e': 1000.0}

Displaying current uptake and secretion

In [18]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
fe2_e,EX_fe2_e,6.333E-05,0,0.00%
fru_e,EX_fru_e,3,6,100.00%
h_e,EX_h_e,10.72,0,0.00%
nh4_e,EX_nh4_e,3.038,0,0.00%
o2_e,EX_o2_e,5.734,0,0.00%
pi_e,EX_pi_e,7.987,0,0.00%
so4_e,EX_so4_e,0.05797,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
BIOMASS_c,EX_BIOMASS_c,-0.2853,0,0.00%
co2_e,EX_co2_e,-6.151,1,100.00%


## Changing media to only fructose as c-source

In [19]:
with model:
    medium = model.medium
    medium ['EX_fru_e'] = 10
    medium ["EX_glycerol_e"] = 0
    model.medium = medium
    model.objective = model.reactions.EX_pbhb_e
    solution = model.optimize()
    print(model.medium)
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    phb_production_fru = model.optimize().objective_value
    print('The current maximum theoretical productivity for making PHB:', phb_production_fru , 'mmol/gDW*h')
    print('----------------')
    maximum_yield = phb_production_fru / (-1*(model.reactions.EX_fru_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol fructose')

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 10, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0}
The current theoretical maximum biomass productivity: 0.0 h
----------------
The current maximum theoretical productivity for making PHB: 12.542372881355982 mmol/gDW*h
----------------
Maximum theoretical yield = 1.2542372881355983 mmol phb/mmol fructose


#### Changing media to only glycerol as c-source

In [20]:
with model:
    medium = model.medium
    medium ['EX_fru_e'] = 0
    medium ["EX_glycerol_e"] = 10
    model.medium = medium
    solution = model.optimize()
    print(model.medium)
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    model.objective = model.reactions.EX_pbhb_e
    phb_production_gly=model.optimize().objective_value
    print('The current maximum theoretical productivity for making PHB:', phb_production_gly , 'mmol/gDW*h')
    print('----------------')
    maximum_yield = phb_production_gly / (-1*(model.reactions.EX_glycerol_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol glycerol')

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_glycerol_e': 10}
The current theoretical maximum biomass productivity: 0.0 h
----------------
The current maximum theoretical productivity for making PHB: 0.0 mmol/gDW*h
----------------


ZeroDivisionError: float division by zero

In [22]:
model.objective = model.reactions.EX_pbhb_e
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
fru_e,EX_fru_e,3,6,100.00%
o2_e,EX_o2_e,1.068,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-2.949,1,100.00%
h2o_e,EX_h2o_e,-6.712,0,0.00%
phb_e,EX_pbhb_e,-3.763,0,0.00%
